In [ ]:
import functools
import operator
import string

import joblib
import textblob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [ ]:
# sns.set(style="whitegrid")
sns.set(style="darkgrid")

In [ ]:
X = pd.read_csv('quora_challenge.csv')

In [ ]:
# the vast majority of questions are short texts
X['question_text'].str.len().describe()

In [ ]:
X['question_len'] = X['question_text'].str.len()

fig, (ax0, ax1) = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))
fig.suptitle('Question Length Distribution', fontsize=16)

sns.violinplot(X['question_len'], inner='quartile', orient='h', ax=ax0)
ax0.set_xlabel('Character Length')

sns.violinplot(X['question_len'], inner='quartile', orient='h', ax=ax1)
ax1.set_xscale('log')
ax1.set_xlabel('Character Length (Log Scale)');

In [ ]:
class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, *, target_col='question_text', result_col='cleaned_text', pos='anvr', stop_words=None):
        self.target_col = str(target_col)
        self.result_col = str(result_col)
        self.pos = tuple(pos.lower()) if isinstance(pos, str) else tuple(pos)
        self.stop_words = () if stop_words is None else frozenset(stop_words)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X[self.result_col] = (X[self.target_col].str.lower()
                                                .map(self.asciitize)
                                                .map(self.depunctuate)
                                                .map(self.lemmatize))
        return X[self.result_col].values
        
    @staticmethod
    def asciitize(text):
        return ''.join(char for char in text if char in string.printable)

    @staticmethod
    def depunctuate(text):
        return ''.join(char if char not in string.punctuation else ' ' for char in text)
    
    def lemmatize(self, text):
        tag_dict = dict(J='a', N='n', V='v', R='r')
        blob = textblob.TextBlob(text)
        
        try:
            words, tags = zip(*blob.pos_tags)
        except ValueError:
            return ()
        
        tags = (tag_dict.get(tag[0]) for tag in tags)
        lemmas = (word.lemmatize(tag) for word, tag in zip(words, tags)
                  if tag in self.pos
                  if word not in self.stop_words)
        return ' '.join(lem for lem in lemmas if lem not in self.stop_words)

In [ ]:
EXTRA_STOP_WORDS = frozenset("does doesn doesnt don dont im ive make quora really shouldnt youll ve weve wouldnt".split())
NOT_STOP_WORDS = frozenset("cry system".split())
CUSTOM_STOP_WORDS = ENGLISH_STOP_WORDS | EXTRA_STOP_WORDS - NOT_STOP_WORDS

In [ ]:
# Split to check if topic labelling seems to make sense later
X_tr, X_te = train_test_split(X, test_size=0.001, random_state=0)
len(X_tr), len(X_te)

In [ ]:
vector_kwds = dict(
    ngram_range=(1, 2),
    stop_words=None,
    lowercase=False,
    max_df=0.9,
    max_features=5_000,
)

decomp_kwds = dict(
    n_components=10,
    random_state=0,
)

topic_model_pipe = Pipeline([
    ('textprep', TextPreprocessor()),
    ('vectorizer', TfidfVectorizer(**vector_kwds)),
    ('decomposer', NMF(**decomp_kwds))
])

In [ ]:
# FIXME
topic_model_pipe.fit(X_te)
file_prefix = '_'.join(type(step).__name__ for step in topic_model_pipe.named_steps.values()).lower()

In [ ]:
joblib.dump(topic_model_pipe, f'{file_prefix}__pipeline.joblib')

In [ ]:
# Get top words for each category -- does not need X
n_top = 8

feature_names = topic_model_pipe['vectorizer'].get_feature_names()
pyfunc = functools.partial(operator.getitem, feature_names)
vfunc = np.vectorize(pyfunc)

components = topic_model_pipe['decomposer'].components_
word_idxs = components.argsort(axis=1)[:, -n_top:][:, ::-1]
words = vfunc(word_idxs)

top_words_df = pd.DataFrame(words.T, columns=[f'Topic {i+1:0>2}' for i in range(len(words))])
top_words_df.index.name = 'Top Words'

In [ ]:
joblib.dump(top_words_df, f'{file_prefix}__top_words.joblib')

In [ ]:
X_new = topic_model_pipe.transform(X_te) # <-- FIXME X

In [ ]:
# Get topic count distribution
topics = np.argmax(X_new, axis=1)
x, y = np.vstack(np.unique(topics, return_counts=True))

plt.figure(figsize=(10, 5))
sns.barplot(x, y, palette='GnBu_d')
plt.title('Topic Distribution')
plt.xticks(ticks=range(len(x)), labels=list('abcdefghijk')) # <-- FIXME labels
plt.xlabel('Topic')
plt.ylabel('Frequency');

In [ ]:
# Get top representative sentences
n_sents = 5
repr_idxs = np.argsort(X_new, axis=0)[-n_sents:].T
repr_sents = X['question_text'].values[repr_idxs]

In [ ]:
joblib.dump(repr_sents, f'{file_prefix}__repr_sents.joblib')

In [ ]:
# FIXME X

# Label sentences to their best topic
X_tr['topic'] = np.argmax(X_new, axis=1)

joblib.dump(X_tr, f'{file_prefix}__X_tr_clean_topic.joblib')